In [1]:
import gensim
from gensim import corpora, models, similarities
import pickle
import multiprocessing
import numpy as np
import pandas as pd
import logging
import warnings

# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
warnings.filterwarnings(action='ignore')

assert gensim.models.doc2vec.FAST_VERSION > -1
cores = multiprocessing.cpu_count()

unlabelled_corpus = pickle.load(open('./input/unlabelled_corpus_clean.p','rb'))
true_corpus = pickle.load(open('./input/true_corpus_clean.p','rb'))
fake_corpus = pickle.load(open('./input/fake_corpus_clean.p','rb'))

C:\Anaconda\envs\py36\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
dictionary = corpora.Dictionary(unlabelled_corpus + true_corpus + fake_corpus)
unlabelled_bow = [dictionary.doc2bow(s) for s in unlabelled_corpus]
true_bow = [dictionary.doc2bow(s) for s in true_corpus]
fake_bow = [dictionary.doc2bow(s) for s in fake_corpus]

In [5]:
num_topics = 100
chunksize = 400
passes = 5

hasNext = True
start = 90000
f = 1000
ctr = 0

# model = models.LdaModel(unlabelled_bow[:start], id2word=dictionary, num_topics=num_topics,alpha = 'auto',eta='auto',random_state=0, chunksize=chunksize, passes=passes)
model = pickle.load(open('./input/lda_model.p','rb'))

In [ ]:
while(hasNext):  
    s = (ctr * f)+start
    e = min(s + f, len(unlabelled_bow))
    print ('REUPDATE:'+ str(s))
    if ((ctr+1) * f) >= len(unlabelled_bow):
        hasNext = False
    model.update(unlabelled_bow[s:e])
    ctr +=1
    pickle.dump(model,open('./input/lda_model.p','wb'))

REUPDATE:90000
REUPDATE:91000
REUPDATE:92000
REUPDATE:93000
REUPDATE:94000
REUPDATE:95000
REUPDATE:96000
REUPDATE:97000
REUPDATE:98000
REUPDATE:99000
REUPDATE:100000
REUPDATE:101000
REUPDATE:102000
REUPDATE:103000
REUPDATE:104000
REUPDATE:105000
REUPDATE:106000
REUPDATE:107000
REUPDATE:108000
REUPDATE:109000
REUPDATE:110000


In [ ]:
def gen_frame(corpus_bow,model,num_topics):
    topic_dists = np.zeros([len(corpus_bow),num_topics])
    
    for i,item in enumerate(corpus_bow):       
        dists = model.get_document_topics(item)        
        indices = list(dict(dists).keys())        
        vals = list(dict(dists).values())        
        topic_dists[i,indices] = vals

    topic_dists = pd.DataFrame(topic_dists, columns = ['topic'+str(a) for a in range(num_topics)])

In [ ]:
unlabelled_lda_vec = gen_frame(unlabelled_bow, model, num_topics)
true_lda_vec = gen_frame(true_bow, model, num_topics)
fake_lda_vec = gen_frame(fake_bow, model, num_topics)

In [ ]:
pickle.dump(unlabelled_lda_vec, open('./input/unlabelled_lda_vec.p','wb'))
pickle.dump(true_lda_vec, open('./input/true_lda_vec.p','wb'))
pickle.dump(fake_lda_vec, open('./input/fake_lda_vec.p','wb'))